In [1]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
from datasets import load_dataset, load_metric

raw_datasets = load_dataset("glue" , "cola")

In [3]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1063
    })
})

In [4]:
from transformers import AutoTokenizer

model_checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [5]:
def preprocess_function(examples):
    return tokenizer(examples["sentence"], truncation=True)

tokenized_datasets = raw_datasets.map(preprocess_function, batched = True)

tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 8551
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1043
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1063
    })
})

In [6]:
from transformers import DataCollatorWithPadding

collator = DataCollatorWithPadding(tokenizer= tokenizer,
                                   return_tensors= 'tf')

train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask","input_ids", "labels", "token_type_ids"],
    collate_fn = collator,
    batch_size = 32,
    shuffle = True
)

validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask","input_ids", "labels", "token_type_ids"],
    collate_fn = collator,
    batch_size = 32,
    shuffle = False
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [7]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint , from_pt= True)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [8]:
from transformers import AdamWeightDecay

optimizer = AdamWeightDecay(2e-5, weight_decay_rate = 0.01)

model.compile(optimizer = optimizer)

In [9]:
from transformers import PushToHubCallback

callbacks = [PushToHubCallback('model_outpus/',
                               tokenizer= tokenizer,
                               hub_model_id= 'bert-fine-tuned-cola')]

model.fit(train_dataset, validation_data = validation_dataset, epochs= 2 , callbacks= callbacks)

Cloning https://huggingface.co/EmincanY/bert-fine-tuned-cola into local empty directory.


Epoch 1/2
268/268 [==============================] - 435s 2s/step - loss: 0.6105 - val_loss: 0.6183
Epoch 2/2
268/268 [==============================] - 414s 2s/step - loss: 0.6125 - val_loss: 0.6181


In [ ]:
# model.push_to_hub('bert-fine-tuned-cola' , commit_message= 'End of training') # 2.push way.

# Labels

In [10]:
label_names = raw_datasets["train"].features["label"].names
label_names

['unacceptable', 'acceptable']

In [11]:
model.config.id2label = {str(i): lbl for i, lbl in enumerate(label_names)}
model.config.label2id = {lbl:str(i) for i, lbl in enumerate(label_names)}

In [12]:
repo_name = "bert-fine-tuned-cola"
model.config.push_to_hub(repo_name)

CommitInfo(commit_url='https://huggingface.co/EmincanY/bert-fine-tuned-cola/commit/ef7b0942acc44a612fd26f87b2f9b94fe90799f9', commit_message='Upload config', commit_description='', oid='ef7b0942acc44a612fd26f87b2f9b94fe90799f9', pr_url=None, pr_revision=None, pr_num=None)

# Load Model

In [13]:
loaded_model = TFAutoModelForSequenceClassification.from_pretrained('EmincanY/bert-fine-tuned-cola')

c:\Users\Emincan\miniconda3\envs\nlp\lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Emincan\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


Some layers from the model checkpoint at EmincanY/bert-fine-tuned-cola were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at EmincanY/bert-fine-tuned-cola.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [ ]:
# Done.